In [1]:
import warnings
# warnings.filterwarnings("ignore")

In [2]:
import datetime
import glob
from functools import lru_cache, partial
from pprint import pprint
import ray

import gym_trading_env
import gymnasium as gym
import matplotlib.pyplot as plt
import MultiTrade
import numpy as np
import pandas as pd
import torch
from gym_trading_env.downloader import download
from gym_trading_env.environments import TradingEnv
from gym_trading_env.renderer import Renderer
from IPython.display import display
from ray import train, tune
from tqdm.autonotebook import tqdm
from utils.utils import build_dataset, build_market_image,preprocess_data,stack_arrays
from utils.forecast_utils import create_ts_preprocessor,create_ts_dataset

from MultiTrade.environments import ForecastingTradingEnv,NeuralForecastingTradingEnv
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from neuralforecast.core import NeuralForecast
from ray.tune.schedulers import HyperBandScheduler

from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.optuna import OptunaSearch

In [3]:
from pearl.pearl_agent import PearlAgent
from pearl.utils.instantiations.environments.gym_environment import GymEnvironment
from pearl.neural_networks.sequential_decision_making.q_value_networks import VanillaQValueNetwork
from pearl.utils.functional_utils.experimentation.set_seed import set_seed
from pearl.policy_learners.sequential_decision_making.deep_q_learning import DeepQLearning
from pearl.policy_learners.sequential_decision_making.double_dqn import DoubleDQN
from pearl.replay_buffers.sequential_decision_making.fifo_off_policy_replay_buffer import FIFOOffPolicyReplayBuffer
from pearl.utils.functional_utils.train_and_eval.online_learning import online_learning
from pearl.action_representation_modules.one_hot_action_representation_module import OneHotActionTensorRepresentationModule

In [4]:
COIN_PAIRS=['BTC/USDT','ETH/USDT','SOL/USDT','BNB/USDT','XRP/USDT','ADA/USDT',
            'ETH/BTC','SOL/ETH','BNB/ETH','XRP/ETH',"ADA/ETH",
            'SOL/BTC','SOL/BNB',
            'XRP/BTC','XRP/BNB',
            'ADA/BTC','ADA/BNB',
            ]
target_pair='ETHUSDT'
time_frame="1h"

In [5]:

# download(exchange_names = ["binance"],
#     symbols= tqdm(COIN_PAIRS),
#     timeframe= time_frame,
#     dir = "data",
#     since= datetime.datetime(year= 2022, month= 1, day=1),
# )

In [6]:
model=NeuralForecast.load("C:/Users/standard/Git/MultiTrader/forecaster_pretrain/neuralForecast")

Seed set to 1
Seed set to 1
Seed set to 1


In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
start_date=datetime.datetime(year= 2024, month= 2, day=1)
split_date=start_date+datetime.timedelta(days=30)
end_date=split_date+datetime.timedelta(days=30)
target_pair='ETH/USDT'
data=build_market_image(target_pair=target_pair,time_frame='1h',axis=0)
data=data[data['symbol']==target_pair.replace('/','')].copy()
data['ds']=data.index
# hf_data=data.copy()

# hf_train_data=hf_data.groupby('symbol').apply(lambda x: x[:split_date])
# hf_train_data=hf_data.groupby('symbol').apply(lambda x: x[:split_date])

# hf_train_data=hf_train_data.reset_index(level=0,drop=True).reset_index()
# prepper=create_ts_preprocessor(hf_train_data)
# hf_train_data=prepper.preprocess(hf_train_data)


In [9]:
split_date=datetime.datetime(year= 2024, month= 3, day=1)
end_date=split_date+datetime.timedelta(days=14)

# data=build_market_image(target_pair='ETH/USDT',time_frame='1h')

hf_data=data.copy()
hf_train_data=hf_data.groupby('symbol').apply(lambda x: x[:split_date])
hf_test_data=hf_data.groupby('symbol').apply(lambda x: x[split_date:end_date])
hf_test_data=hf_test_data.reset_index(level=0,drop=True).reset_index()
# hf_test_data

In [10]:
def get_train_test_envs(data,symbol='ETHUSDT',look_back=7):
    start_date=datetime.datetime(year= 2024, month= 1, day=1)
    split_date=datetime.datetime(year= 2024, month= 3, day=1)
    end_date=split_date+datetime.timedelta(days=14)

    
    hf_data=data.copy()
    
    hf_train_data=hf_data.groupby('symbol').apply(lambda x: x[:split_date])
    hf_test_data=hf_data.groupby('symbol').apply(lambda x: x[split_date:end_date])

    hf_train_data=hf_train_data.reset_index(level=0,drop=True).reset_index()
    hf_test_data=hf_test_data.reset_index(level=0,drop=True).reset_index()


    trade_data=data[data['symbol']==symbol].copy()
    train_data=trade_data[:split_date]
    test_data=trade_data[split_date:end_date]


    train_env = NeuralForecastingTradingEnv(
                                        model=model,

                                        name='ETHUSDT_train',
                                        df = train_data, # Your dataset with your custom features
                                        positions = [ -.5, 0, .5], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
                                        trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
                                        borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
                                        max_episode_duration=168,
                                        verbose=0

                                        )
    
    test_env = NeuralForecastingTradingEnv(
                                        model=model,
                                        # hf_data=hf_test_data,
                                        
                                        name='ETHUSDT_test',
                                        df = test_data, # Your dataset with your custom features
                                        positions = [ -.5, 0, .5], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
                                        trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
                                        borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
                                        max_episode_duration=168,
                                        # verbose=0
                                        
                                    )
    return train_env,test_env

In [11]:



train_env,test_env=get_train_test_envs(data,look_back=7)
train_pearl_env=GymEnvironment(train_env)
test_pearl_env=GymEnvironment(test_env)
# obs,info=test_env.reset()
# obs

337 331 330


In [12]:
obs=train_env.reset()
# env.action_space.sa
done=False
while not done:
    act=train_pearl_env.action_space.sample()
    act_result=train_pearl_env.step(act)
    done=act_result.done
# act_result.info

In [13]:
# obs=train_env.reset()
# # env.action_space.sa
# for i in range(20):
#     act=train_env.action_space.sample()
#     act_result=train_env.step(act)
# act_result


In [14]:
search_space={
        # "look_back" : tune.choice([7,14,21,30,45,60]),

        "hidden_dims" : tune.choice([[64,64],[128,128],[256,256]]),

        'learning_rate':tune.uniform(1e-6, 1e-2),

        'discount_factor': tune.uniform(.5, 1),

        'training_rounds': tune.choice([c for c in range(2,64,2)]),

        'batch_size': tune.choice([64,128,256]),
        
        'target_update_freq':tune.choice([c for c in range(2,64,2)]),

        'soft_update_tau': tune.uniform(1e-6, 1),  # a value of 1 indicates no soft updates
        
        "replay_buffer_size":tune.choice([c for c in range(10,200,10)]),
        }
search_space

{'hidden_dims': <ray.tune.search.sample.Categorical at 0x1992da61a00>,
 'learning_rate': <ray.tune.search.sample.Float at 0x1992da61a90>,
 'discount_factor': <ray.tune.search.sample.Float at 0x1992da61b20>,
 'training_rounds': <ray.tune.search.sample.Categorical at 0x1992da61bb0>,
 'batch_size': <ray.tune.search.sample.Categorical at 0x1992da61c40>,
 'target_update_freq': <ray.tune.search.sample.Categorical at 0x1992d12fa30>,
 'soft_update_tau': <ray.tune.search.sample.Float at 0x1992d12f580>,
 'replay_buffer_size': <ray.tune.search.sample.Categorical at 0x1992d12f730>}

In [15]:
def plot_pearl(pearl_env):
    naked_env=pearl_env.env.unwrapped
    value_history=naked_env.historical_info['portfolio_valuation']
    x=np.arange(len(value_history))
    y=value_history
    plt.plot(x,y)


In [16]:


def objective(config):
    import warnings
    warnings.filterwarnings('ignore')
    warnings.simplefilter('ignore')
    hidden_dims=list(config["hidden_dims"])
    replay_buffer_size=config["replay_buffer_size"]


    # Instead of using the 'network_type' argument, use the 'network_instance' argument.
    # Pass Q_value_network as the `network_instance` to the `DeepQLearning` policy learner.
    # We will be using a one hot representation for representing actions. So take action_dim = num_actions.
    Q_network_DoubleDQN = VanillaQValueNetwork(state_dim=train_env.observation_space.shape[0],  # dimension of the state representation
                                                action_dim=train_env.action_space.n,                        # dimension of the action representation
                                                hidden_dims=hidden_dims,                       # dimensions of the intermediate layers
                                                output_dim=1)  
    # Instead of using the 'network_type' argument, use the 'network_instance' argument.
    # Pass Q_value_network as the `network_instance` to the `DoubleDQN` policy learner.
    DoubleDQNagent = PearlAgent(
                                policy_learner=DoubleDQN(
                                                            state_dim=train_env.observation_space.shape[0],
                                                            action_space=train_env.action_space,
                                                            network_instance=Q_network_DoubleDQN,   # pass an instance of Q value network to the policy learner.
                                                            action_representation_module=OneHotActionTensorRepresentationModule(
                                                                                                                                    max_number_actions=train_env.action_space.n
                                                                                                                                ),
                                                                                                                                
                                                            **config
                                                        ),
                                replay_buffer=FIFOOffPolicyReplayBuffer(replay_buffer_size),
                            ) 
    ## train dat bitch               
    info = online_learning(
                            agent=DoubleDQNagent ,
                            env=train_pearl_env,
                            number_of_episodes=2_000,
                            print_every_x_episodes=500,   # print returns after every 10 episdoes
                            learn_after_episode=True,    # updating after every environment interaction, Q networks are updates at the end of each episode
                            seed=0
                            )
    # plot_results(info)
    agent=DoubleDQNagent
    observation, action_space = test_pearl_env.reset()
    agent.reset(observation, action_space)
    done = False
    while not done:
        action = agent.act(exploit=True)
        action_result = test_pearl_env.step(action)
        agent.observe(action_result)
        agent.learn()
        done = action_result.done

    # plot_pearl(test_env)
    score=action_result.info['portfolio_valuation']-1000
    rew=action_result.info['reward']
    
    loss={"score": score,
          "reward": rew,
            "_metric": score}
    print(loss)
    return loss

In [17]:
config={x:y.sample() for x,y in search_space.items()}

# objective(config)

In [18]:
algo = OptunaSearch()
algo = ConcurrencyLimiter(algo, max_concurrent=4)

In [19]:

def trial_str_creator(trial):
    return "{}_{}_trading_agent".format(trial.trainable_name, trial.trial_id)

hyperband = HyperBandScheduler(metric="score", mode="max")

tune_config=tune.TuneConfig(num_samples=100,
                            mode="max",
                            # search_alg='optuna',
                            search_alg=algo,
                                trial_name_creator=trial_str_creator,
                                trial_dirname_creator=trial_str_creator,
                                # scheduler=hyperband,
                            )
run_config=train.RunConfig(
                            storage_path='C:/Users/standard/OneDrive/Documents/Git/MultiTrader/tune_results', 
                            name="DDQN_experiments",
                            
                            
                            )
scaling_config=train.ScalingConfig(num_workers=4
                                   
                                   )
objective_with_resources = tune.with_resources(objective, {"cpu": 0.5})
# objective_with_resources = tune.with_resources(objective,{"cpu": 8,})
objective_with_resources = tune.with_resources(objective,{"cpu": 8,'gpu':1})



In [20]:
%%writefile custom.css
/* custom.css */

/* Set background color for all widgets */
.tuneStatus {
    /* white background */
    background-color: #ffffff;
}

Overwriting custom.css


In [21]:
from IPython.display import display, HTML

# Load custom CSS file
css = HTML('<link rel="stylesheet" type="text/css" href="./custom.css">') # make sure the path is correct, it fails silently otherwise
display(css)

In [22]:

tuner = tune.Tuner(objective_with_resources ,
                   tune_config=tune_config,
                   run_config=run_config,
                #    scaling_config=scaling_config,
                    
                    param_space=search_space)  




In [23]:
tuner.fit()

(raylet) Warning: The actor ImplicitFunc is very large (52 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(objective pid=10628) episode 100, step 16700, agent=PearlAgent with DoubleDQN, FIFOOffPolicyReplayBuffer, env=CustomGymEnvironment
(objective pid=10628) return: -0.008408576770307263
(objective pid=10628) episode 200, step 33400, agent=PearlAgent with DoubleDQN, FIFOOffPolicyReplayBuffer, env=CustomGymEnvironment
(objective pid=10628) return: 0.0017522163943795022
(objective pid=10628) episode 300, step 50100, agent=PearlAgent with DoubleDQN, FIFOOffPolicyReplayBuffer, env=CustomGymEnvironment
(objective pid=10628) return: -0.008924095203838078
(objective pid=10628) episode 400, step 66800, agent=PearlAgent with DoubleDQN, FIFOOffPolicyReplayBuffer, env=CustomGymEnvironment
(objective pid=10628) return: 0.021158812884095823
(objective pid=10628) episode 500, step 8

In [ ]:
results = tuner.fit
df_results = results.get_dataframe()
df_results

In [ ]:
best_params=results.get_best_result(metric="score", mode="max").config
best_params
# objective(best_params)

In [ ]:
test_env.env.unwrapped.save_for_render(dir = "test_render_logs")

In [ ]:
import tensorboard

In [ ]:

renderer = Renderer(render_logs_dir = "test_render_logs")

In [ ]:
# Add Custom Metrics (Annualized metrics)
renderer.add_metric(
    name = "Annual Market Return",
    function = lambda df : f"{ ((df['close'].iloc[-1] / df['close'].iloc[0])**(pd.Timedelta(days=365)/(df.index.values[-1] - df.index.values[0]))-1)*100:0.2f}%"
)
renderer.add_metric(
        name = "Annual Portfolio Return",
        function = lambda df : f"{((df['portfolio_valuation'].iloc[-1] / df['portfolio_valuation'].iloc[0])**(pd.Timedelta(days=365)/(df.index.values[-1] - df.index.values[0]))-1)*100:0.2f}%"
)

renderer.run()